In [ ]:
#!/ usr/bin/env python3 .7
# Copyright 2020 , Gurobi Optimization , LLC

# This example formulates and solves the following simple model
# with PWL constraints :
#
# maximize
#     sum c[j] * x[j]
# subject to
#     sum A[i,j] * x[j] <= 0,     for i = 0, ... , m -1
#     sum y[j] <= 3
#     y[j] = pwl(x[j]),           for j = 0, ... , n -1
#     x[j] free , y[j] >= 0,      for j = 0, ... , n -1
# where pwl(x) = 0,      if x = 0
#              = 1+|x|,  if x != 0

# Note
# 1. sum pwl(x[j]) <= b is to bound x vector and also to favor sparse x vector .
#    Here b = 3 means that at most two x[j] can be nonzero and if two , then
#    sum x[j] <= 1
# 2. pwl(x) jumps from 1 to 0 and from 0 to 1, if x moves from negative 0 to 0,
#    then to positive 0, so we need three points at x = 0. x has infinite bounds
#    on both sides , the piece defined with two points (-1, 2) and (0, 1) can
#    extend x to -infinite . Overall we can use five points (-1, 2), (0, 1),
#    (0, 0), (0, 1) and (1, 2) to define y = pwl(x)

A piecewise-linear (PWL) constraint states that the relationship `y = f(x)` must hold between variables `x` and `y`, where `f` is a piecewise-linear function.

In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
n = 5
m = 5
c = [0.5, 0.8, 0.5, 0.1, -1]
A = [[0, 0, 0, 1, -1],
     [0, 0, 0, 1, -1],
     [1, 1, 0, 0, -1],
     [1, 0, 1, 0, -1],
     [1, 0, 0, 1, -1]
    ]

In [3]:
# Create a new model
model = gp.Model('gc_pwl')

Using license file /Users/yj/gurobi.lic


In [4]:
# Create variables
x = model.addVars(n, lb=-GRB.INFINITY, name='x')
y = model.addVars(n, name='y')

In [5]:
# Set objective
model.setObjective(gp.quicksum(c[j]*x[j] for j in range(n)), GRB.MAXIMIZE)

In [6]:
# Add Constraints 1
for i in range(m):
    model.addConstr(gp.quicksum(A[i][j]*x[j] for j in range(n)) <= 0)

In [7]:
# Add Constraints 2
model.addConstr(y.sum() <= 3)

<gurobi.Constr *Awaiting Model Update*>

In [8]:
# Add Constraints 3
for j in range(n):
    model.addGenConstrPWL(x[j], y[j], [-1, 0, 0, 0, 1], [2, 1, 0, 1, 2])

In [10]:
# Optimize model
model.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 6 rows, 10 columns and 18 nonzeros
Model fingerprint: 0x8c73bd71
Model has 5 general constraints
Variable types: 10 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective -0.0000000
Presolve added 9 rows and 20 columns
Presolve time: 0.02s
Presolved: 15 rows, 30 columns, 64 nonzeros
Presolved model has 10 SOS constraint(s)
Variable types: 30 continuous, 0 integer (0 binary)

Root relaxation: objective 8.000000e-01, 3 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.80000    0    3   -0.00000    0.80000      -     -    0s
     0     0    0.80000    0    3   -0.00000    0.80000      -     -    0s
 

In [11]:
for j in range(n):
    print('%s = %g' % (x[j].varName, x[j].x))

print('Obj: %g' % model.objVal )

x[0] = -0.5
x[1] = 0.5
x[2] = 0
x[3] = 0
x[4] = 0
Obj: 0.15
